## Summary

Calculate features using [ProtBert](https://github.com/agemagician/ProtTrans).

```bash
export NOTEBOOK_PATH="$(realpath 31_run_rosetta_ddg.ipynb)"

export DATASET_NAME="cagi6-sherloc"
export DATASET_PATH="30_cagi6_sherloc/input-data-gby-protein.parquet"
export ORIGINAL_ARRAY_TASK_COUNT=4182

sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=1-1 --time 3:00:00 --mem=0 --exclusive ../scripts/run_notebook_cpu.sh


# Niagara
sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=1-4182 --time 3:00:00 --mem=0 --exclusive ../scripts/run_notebook_cpu.sh
```


```bash
export NOTEBOOK_PATH="$(realpath 31_run_rosetta_ddg.ipynb)"
export DATASET_NAME="humsavar"
export DATASET_PATH="30_humsavar/humsavar-gby-protein.parquet"
export ORIGINAL_ARRAY_TASK_COUNT=612

sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=1-612 --time 24:00:00 --ntasks-per-node=10 --mem=20G ../scripts/run_notebook_cpu.sh
```

---

## Imports

In [ ]:
import concurrent.futures
import itertools
import os
import re
import socket
import subprocess
import sys
import tempfile
from pathlib import Path

import kmbio
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from elaspic2.plugins.rosetta_ddg import RosettaDDG
from kmbio import PDB
from tqdm.notebook import tqdm

## Parameters

In [ ]:
NOTEBOOK_DIR = Path("31_run_rosetta_ddg").resolve()
NOTEBOOK_DIR.mkdir(exist_ok=True)

NOTEBOOK_DIR

In [ ]:
if (slurm_tmpdir := os.getenv("SLURM_TMPDIR")) is not None:
    os.environ["TMPDIR"] = slurm_tmpdir
    
print(tempfile.gettempdir())

In [ ]:
if "scinet" in socket.gethostname():
    CPU_COUNT = 40
else:
    CPU_COUNT = max(1, len(os.sched_getaffinity(0)))

# CPU_COUNT = max(1, CPU_COUNT // 2)

CPU_COUNT

In [ ]:
DATASET_NAME = os.getenv("DATASET_NAME")
DATASET_PATH = os.getenv("DATASET_PATH")
TASK_ID = os.getenv("SLURM_ARRAY_TASK_ID")
TASK_ID_OFFSET = int(os.getenv("ARRAY_TASK_ID_OFFSET", "0"))
TASK_COUNT = os.getenv("ORIGINAL_ARRAY_TASK_COUNT") or os.getenv("SLURM_ARRAY_TASK_COUNT")

TASK_ID = (int(TASK_ID) + TASK_ID_OFFSET) if TASK_ID is not None else None
TASK_COUNT = int(TASK_COUNT) if TASK_COUNT is not None else None

DATASET_NAME, DATASET_PATH, TASK_ID, TASK_ID_OFFSET, TASK_COUNT

In [ ]:
DEBUG = TASK_ID is None

if DEBUG:
    DATASET_NAME = "humsavar"
    DATASET_PATH = str(
        NOTEBOOK_DIR.parent.joinpath("30_humsavar", "humsavar-gby-protein.parquet")
        # NOTEBOOK_DIR.parent.joinpath("30_cagi6_sherloc", "input-data-gby-protein.parquet")
    )
    TASK_ID = 612
    TASK_COUNT = 612 # 4182
else:
    assert DATASET_NAME is not None
    assert DATASET_PATH is not None
    assert TASK_ID is not None
    assert TASK_COUNT is not None
    DATASET_PATH = Path(DATASET_PATH).expanduser().resolve()

DATASET_NAME, DATASET_PATH, TASK_ID, TASK_COUNT

In [ ]:
output_file = NOTEBOOK_DIR.joinpath(DATASET_NAME, f"shard-{TASK_ID}-of-{TASK_COUNT}.parquet")
output_file.parent.mkdir(exist_ok=True)

output_file

In [ ]:
if output_file.is_file():
    raise Exception("Already finished!")

## Workspace

### Load data

In [ ]:
pfile = pq.ParquetFile(DATASET_PATH)

pfile.num_row_groups

In [ ]:
assert TASK_COUNT == pfile.num_row_groups

In [ ]:
input_df = pfile.read_row_group(TASK_ID - 1).to_pandas(integer_object_nulls=True)

In [ ]:
display(input_df.head(2))
print(len(input_df))

In [ ]:
protein_id_column = None

for col in ["protein_id", "uniprot_id"]:
    if col in input_df:
        protein_id_column = col
        
assert protein_id_column is not None
protein_id_column

In [ ]:
tup = next(input_df.itertuples(index=False))

iterable_fields = []
for field in tup._fields:
    if field in [protein_id_column]:
        continue
    try:
        if len(getattr(tup, field)) == len(tup.mutation):
            iterable_fields.append(field)
    except TypeError:
        pass

iterable_fields

In [ ]:
if DEBUG:
    for field in iterable_fields:
        input_df[field] = input_df[field].str[:3]

### Score mutations

In [ ]:
def validate_mutation(mutation):
    aa = "GVALICMFWPDESTYQNKRH"
    if re.search(f"^[{aa}][1-9]+[0-9]*[{aa}]$", mutation) is None:
        print(f"Skipping mutation {mutation} because it appears to be malformed.")
        return False

    if mutation[0] == mutation[-1]:
        print(
            f"Skipping mutation {mutation} because the wildtype and mutant residues are the same."
        )
        return False

    return True

In [ ]:
def worker(idx, mutation_idx, mutation, data):
    if not validate_mutation(mutation):
        return None

    try:
        results = RosettaDDG.analyze_mutation(f"A_{mutation}", data)
    except Exception as error:
        print(f"{error!r}")
        return None
    else:
        results = {f"rosetta_{key}": value for key, value in results.items()}
        return idx, mutation_idx, mutation, results

In [ ]:
futures = []
results = []
with concurrent.futures.ProcessPoolExecutor(CPU_COUNT) as pool:
    num_mutations = 0
    for tup in input_df.itertuples():
        assert all(
            [
                (len(getattr(tup, field)) == len(tup.mutation))
                for field in iterable_fields
            ]
        )

        with tempfile.NamedTemporaryFile(suffix=".pdb") as tmp_file:
            with open(tmp_file.name, "wt") as fout:
                fout.write(tup.structure)
            data = RosettaDDG.build(
                tmp_file.name,
                protocol="cartesian_ddg",
                energy_function="beta_nov16_cart",
                interface=0,
            )

        for mutation_idx, mutation in enumerate(tup.mutation):
            future = pool.submit(worker, tup.Index, mutation_idx, mutation, data)
            futures.append(future)
            num_mutations += 1

    for future in tqdm(concurrent.futures.as_completed(futures), total=num_mutations):
        result = future.result()
        if result is None:
            continue

        idx, mutation_idx, mutation, rosetta_result = result
        row = input_df.loc[idx]

        results.append(
            {
                "protein_id": row[protein_id_column],
                "mutation": mutation,
            }
            | {field: row[field][mutation_idx] for field in iterable_fields}
            | rosetta_result
        )

results_df = pd.DataFrame(results)

In [ ]:
display(results_df.head(2))
print(len(results_df))

### Save results

In [ ]:
if not DEBUG:
    pq.write_table(pa.Table.from_pandas(results_df, preserve_index=False), output_file)